# Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np
from stock_prices_predictions.get_data import get_lstm_training_data, get_lstm_test_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras import layers

# Fetch data

In [2]:
X, y = get_lstm_training_data('T', 1)
X_test, y_test = get_lstm_test_data('T', 1)

# Create model

In [3]:
X.shape

(100, 36, 4)

In [ ]:
X[0].shape

In [6]:
type(y)

numpy.ndarray

In [9]:
y

array([14.60025597, 17.62719917, 19.17797661, 17.67354202, 18.92109108,
       23.97682571, 21.051548  , 20.88384247, 18.23783302, 17.61681175,
       18.76831245, 20.04362869, 16.00559807, 17.09865761, 16.78026581,
       13.55665207, 14.15041161, 11.4088335 ,  9.87454319, 10.80444431,
       12.85858917,  9.27087212,  9.86722374, 11.46046543, 10.62842941,
       10.08396626, 10.93715668, 12.92984867, 11.94101143, 12.05215263,
       11.61087227, 12.15259743, 12.48428726, 12.7071991 , 12.35317802,
       11.97883701, 11.58151913, 11.93096638, 12.51780605, 12.06243134,
       12.25264645, 13.00551414, 14.70008087, 13.4355278 , 14.29859829,
       16.49498177, 17.76259422, 17.46606636, 18.50428963, 19.42324638,
       21.47727013, 22.10281754, 22.39568901, 23.70755768, 22.07864571,
       22.20934296, 19.65582657, 22.0122757 , 20.59984589, 17.79920769,
       17.297575  , 15.67885208, 15.85585594, 13.66884422, 15.18669033,
       14.61930847, 15.68643856, 16.84066963, 16.38201332, 17.06

In [4]:
model = Sequential()
model.add(LSTM(units=36, activation='relu', input_shape= (36, 4), return_sequences=True))
model.add(LSTM(units=36, activation='relu'))
model.add(Dense(units=1, activation="linear"))

# Fit model


In [11]:
model.compile(optimizer="rmsprop", loss="mse", metrics="mae")
model.fit(X, y, epochs=500, batch_size=16)

Epoch 1/500
7/7 [==============================] - 3s 63ms/step - loss: 990027008.0000 - mae: 3641.0654
Epoch 2/500
7/7 [==============================] - 0s 71ms/step - loss: 66155.9219 - mae: 89.8693
Epoch 3/500
7/7 [==============================] - 1s 81ms/step - loss: 4218814720.0000 - mae: 7898.3208
Epoch 4/500
7/7 [==============================] - 1s 85ms/step - loss: 17767070.0000 - mae: 1448.4855
Epoch 5/500
7/7 [==============================] - 1s 78ms/step - loss: 85039672.0000 - mae: 1803.0739
Epoch 6/500
7/7 [==============================] - 1s 80ms/step - loss: 3441020.5000 - mae: 783.8312
Epoch 7/500
7/7 [==============================] - 1s 96ms/step - loss: 2382466.0000 - mae: 501.5465
Epoch 8/500
7/7 [==============================] - 1s 85ms/step - loss: 5994552.5000 - mae: 519.9254
Epoch 9/500
7/7 [==============================] - 1s 89ms/step - loss: 3894810.5000 - mae: 432.3449
Epoch 10/500
7/7 [==============================] - 1s 106ms/step - loss: 23769.755

7/7 [==============================] - 0s 69ms/step - loss: 329.1112 - mae: 17.4093
Epoch 85/500
7/7 [==============================] - 0s 67ms/step - loss: 328.8687 - mae: 17.4024
Epoch 86/500
7/7 [==============================] - 0s 64ms/step - loss: 328.6237 - mae: 17.3954
Epoch 87/500
7/7 [==============================] - 0s 64ms/step - loss: 328.3834 - mae: 17.3884
Epoch 88/500
7/7 [==============================] - 0s 66ms/step - loss: 328.1390 - mae: 17.3814
Epoch 89/500
7/7 [==============================] - 0s 64ms/step - loss: 327.8976 - mae: 17.3744
Epoch 90/500
7/7 [==============================] - 0s 68ms/step - loss: 327.6581 - mae: 17.3676
Epoch 91/500
7/7 [==============================] - 0s 64ms/step - loss: 327.4099 - mae: 17.3604
Epoch 92/500
7/7 [==============================] - 0s 62ms/step - loss: 327.1662 - mae: 17.3535
Epoch 93/500
7/7 [==============================] - 0s 62ms/step - loss: 326.9253 - mae: 17.3464
Epoch 94/500
7/7 [=========================

7/7 [==============================] - 0s 65ms/step - loss: 309.3051 - mae: 16.8310
Epoch 168/500
7/7 [==============================] - 0s 64ms/step - loss: 309.0693 - mae: 16.8239
Epoch 169/500
7/7 [==============================] - 0s 63ms/step - loss: 308.8400 - mae: 16.8170
Epoch 170/500
7/7 [==============================] - 0s 64ms/step - loss: 308.6015 - mae: 16.8101
Epoch 171/500
7/7 [==============================] - 0s 64ms/step - loss: 308.3639 - mae: 16.8030
Epoch 172/500
7/7 [==============================] - 0s 65ms/step - loss: 308.1308 - mae: 16.7960
Epoch 173/500
7/7 [==============================] - 0s 63ms/step - loss: 307.8994 - mae: 16.7893
Epoch 174/500
7/7 [==============================] - 0s 64ms/step - loss: 307.6605 - mae: 16.7821
Epoch 175/500
7/7 [==============================] - 0s 63ms/step - loss: 307.4264 - mae: 16.7751
Epoch 176/500
7/7 [==============================] - 0s 66ms/step - loss: 307.1981 - mae: 16.7682
Epoch 177/500
7/7 [===============

7/7 [==============================] - 1s 90ms/step - loss: 290.1799 - mae: 16.2529
Epoch 251/500
7/7 [==============================] - 1s 87ms/step - loss: 289.9515 - mae: 16.2458
Epoch 252/500
7/7 [==============================] - 1s 90ms/step - loss: 289.7237 - mae: 16.2388
Epoch 253/500
7/7 [==============================] - 1s 79ms/step - loss: 289.4959 - mae: 16.2317
Epoch 254/500
7/7 [==============================] - 1s 72ms/step - loss: 289.2737 - mae: 16.2249
Epoch 255/500
7/7 [==============================] - 1s 73ms/step - loss: 289.0455 - mae: 16.2179
Epoch 256/500
7/7 [==============================] - 1s 73ms/step - loss: 288.8180 - mae: 16.2109
Epoch 257/500
7/7 [==============================] - 1s 74ms/step - loss: 288.5935 - mae: 16.2039
Epoch 258/500
7/7 [==============================] - 0s 71ms/step - loss: 288.3647 - mae: 16.1969
Epoch 259/500
7/7 [==============================] - 1s 80ms/step - loss: 288.1394 - mae: 16.1899
Epoch 260/500
7/7 [===============

7/7 [==============================] - 1s 86ms/step - loss: 271.7125 - mae: 15.6743
Epoch 334/500
7/7 [==============================] - 1s 85ms/step - loss: 271.4974 - mae: 15.6676
Epoch 335/500
7/7 [==============================] - 1s 89ms/step - loss: 271.2764 - mae: 15.6604
Epoch 336/500
7/7 [==============================] - 1s 96ms/step - loss: 271.0576 - mae: 15.6535
Epoch 337/500
7/7 [==============================] - 1s 84ms/step - loss: 270.8445 - mae: 15.6466
Epoch 338/500
7/7 [==============================] - 1s 78ms/step - loss: 270.6314 - mae: 15.6399
Epoch 339/500
7/7 [==============================] - 0s 72ms/step - loss: 270.4143 - mae: 15.6330
Epoch 340/500
7/7 [==============================] - 1s 74ms/step - loss: 270.1967 - mae: 15.6260
Epoch 341/500
7/7 [==============================] - 0s 68ms/step - loss: 269.9754 - mae: 15.6189
Epoch 342/500
7/7 [==============================] - 1s 77ms/step - loss: 269.7536 - mae: 15.6118
Epoch 343/500
7/7 [===============

7/7 [==============================] - 1s 78ms/step - loss: 253.9319 - mae: 15.0964
Epoch 417/500
7/7 [==============================] - 1s 92ms/step - loss: 253.7255 - mae: 15.0898
Epoch 418/500
7/7 [==============================] - 1s 110ms/step - loss: 253.5073 - mae: 15.0826
Epoch 419/500
7/7 [==============================] - 1s 116ms/step - loss: 253.2943 - mae: 15.0754
Epoch 420/500
7/7 [==============================] - 1s 106ms/step - loss: 253.0831 - mae: 15.0685
Epoch 421/500
7/7 [==============================] - 1s 122ms/step - loss: 252.8749 - mae: 15.0615
Epoch 422/500
7/7 [==============================] - 1s 111ms/step - loss: 252.6693 - mae: 15.0547
Epoch 423/500
7/7 [==============================] - 1s 132ms/step - loss: 252.4618 - mae: 15.0478
Epoch 424/500
7/7 [==============================] - 1s 103ms/step - loss: 252.2531 - mae: 15.0409
Epoch 425/500
7/7 [==============================] - 1s 128ms/step - loss: 252.0422 - mae: 15.0339
Epoch 426/500
7/7 [=======

7/7 [==============================] - 0s 67ms/step - loss: 236.8295 - mae: 14.5191
Epoch 500/500
7/7 [==============================] - 0s 72ms/step - loss: 236.6269 - mae: 14.5122


# Test Model

In [12]:
res = model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 293ms/step - loss: 942.6374 - mae: 30.4686


In [20]:
res2 = model.predict(X_test)

In [21]:
res2

array([[3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158],
       [3.3622158]], dtype=float32)